In [7]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.dish_bin.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

from utils import get_observed_node_sets

import meshcat
import meshcat.geometry as meshcat_geom

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
print(len(BowlModels), " bowl models")
print(len(CupModels), " cup models")
print(len(PlateModels), " plate models")

3  bowl models
3  cup models
4  plate models


In [21]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

Meshcat url:  http://127.0.0.1:7005/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [22]:
# Load in datasets.
observed_node_sets, observed_node_sets_outliers = get_observed_node_sets(reconvert=True)

DEBUG:root:Saving observed_node_sets.dat
DEBUG:root:Loading observed_node_sets.dat
DEBUG:root:Saving observed_outlier_node_sets.dat
DEBUG:root:Loading observed_outlier_node_sets.dat


In [55]:
# Pick a scene to parse, and draw it.
# Has good view: 1, 4, 5, 6, 10, 11
example_observed_scene = observed_node_sets[17]
draw_scene_tree_contents_meshcat(
    SceneTree.make_from_observed_nodes(example_observed_scene),
    zmq_url=vis.window.zmq_url, prefix="observed/contents"
)


supertree = grammar.make_super_tree(max_recursion_depth=10)
print("Supertree size ", len(supertree.nodes))

Supertree size  613


In [53]:
# Parse it with IP.
vis["ip_parses"].delete()

pyro.set_rng_seed(42)
grammar = SpatialSceneGrammar(
    root_node_type = DishBin,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.5, 0., 0.]))
)
# Build up a big set of candidate intermediate nodes with both
# top-down and bottom-up generation.
proposed_poses_by_type = generate_candidate_node_pose_sets(
    grammar, example_observed_scene, max_recursion_depth=10, verbose=True
)
parse_trees = infer_mle_tree_with_mip_from_proposals(
    grammar, example_observed_scene, proposed_poses_by_type, verbose=1, N_solutions=4
)
parse_tree = parse_trees[0]
for k, parse_tree in enumerate(parse_trees[:1]):
    score = parse_tree.score()
    print("SOL #%d: SCORE %f" % (k, score))
    draw_scene_tree_structure_meshcat(parse_tree, zmq_url=vis.window.zmq_url, prefix="ip_parses/%d_%f" % (k, score), linewidth=20, with_triad=False, node_sphere_size=0.025, alpha=0.5)

192 top-down candidates.
37 bottom-up candidates.
Post final pruning:  229
Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Mon Jan 10 18:25:13 2022

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2966 rows, 1551 columns and 10193 nonzeros
Model fingerprint: 0xefbc993a
Variable types: 0 continuous, 1551 integer (1551 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [3e-01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+00]
Presolve removed 2957 rows and 1540 columns
Presolve time: 0.01s
Presolved: 9 rows, 11 columns, 31 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Found heuristic solution: objective 13.7751565

Root relaxation: objective 1.377516e+01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          

In [29]:
# Parse it with MICP.
trees = get_optimized_trees_from_mip_results(infer_mle_tree_with_mip(grammar, example_observed_scene, N_solutions=4, max_recursion_depth=10, verbose=True))
vis["mip_parses"].delete()
for k, tree in enumerate(trees[:]):
    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="mip_parses/%d_%f" % (k, tree.score()), linewidth=20, with_triad=False, node_sphere_size=0.025, alpha=0.5)

Starting setup.
Activation vars allocated.
Continuous variables and SO(3) constraints allocated for all equivalence sets.
Setup time:  0.9902973175048828
Num vars:  2582
Num constraints:  8174
Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Mon Jan 10 18:20:48 2022

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 7414 rows, 2582 columns and 25836 nonzeros
Model fingerprint: 0xd1155bc5
Model has 126 quadratic objective terms
Variable types: 1842 continuous, 740 integer (740 binary)
Coefficient statistics:
  Matrix range     [6e-07, 2e+01]
  Objective range  [1e-02, 4e+01]
  QObjective range [5e+01, 4e+02]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 4933 rows and 1376 columns
Presolve time: 0.05s
Presolved: 2481 rows, 1206 columns, 10561 nonzeros
Presolved model has 113 quadratic objective terms
Variable types: 1196 continuous, 10 integer (10 binary)

Root relaxation: objective 3.959546